In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
# loding the data
datasets, info= tfds.load(name='cassava', with_info=True, as_supervised=True ,split=['train','test', 'validation'])
info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5656 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteQGE9YB/cassava-test.tfrecord*...:   0%|          |…

Generating validation examples...:   0%|          | 0/1889 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteQGE9YB/cassava-validation.tfrecord*...:   0%|     …

Dataset cassava downloaded and prepared to /root/tensorflow_datasets/cassava/0.1.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='cassava',
    full_name='cassava/0.1.0',
    description="""
    Cassava consists of leaf images for the cassava plant depicting healthy and
    four (4) disease conditions; Cassava Mosaic Disease (CMD), Cassava Bacterial
    Blight (CBB), Cassava Greem Mite (CGM) and Cassava Brown Streak Disease (CBSD).
    Dataset consists of a total of 9430 labelled images.
    The 9430 labelled images are split into a training set (5656), a test set(1885)
    and a validation set (1889). The number of images per class are unbalanced with
    the two disease classes CMD and CBSD having 72% of the images.
    """,
    homepage='https://www.kaggle.com/c/cassava-disease/overview',
    data_path=PosixGPath('/tmp/tmpp_3dz69ktfds'),
    file_format=tfrecord,
    download_size=1.26 GiB,
    dataset_size=1.26 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': Cl

In [18]:
# scaling the data
def scaling (image, label):
  image = tf.cast(image, tf.float32)
  image /=255.0
  return tf.image.resize(image, [224, 224]), tf.one_hot(label, 3)

# train_dataset = train_dataset.map(scaling)


In [19]:
def get_dataset(batch_size=32):
  train_dataset_scaled =datasets[0].map(scaling).shuffle(1000).batch(batch_size)
  test_dataset_scaled =datasets[1].map(scaling).batch(batch_size)
  val_dataset_scaled =datasets[2].map(scaling).batch(batch_size)
  return train_dataset_scaled, test_dataset_scaled, val_dataset_scaled

In [20]:
train_dataset, test_dataset, val_dataset = get_dataset()
train_dataset.cache()
val_dataset.cache()

<CacheDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

In [21]:
len(list(datasets[0]))

5656

In [22]:
# downloading the pretrained model
feature_extractor = "http://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

In [23]:
feature_extractor_layer=hub.KerasLayer(feature_extractor, input_shape=(224, 224,3))

In [24]:
feature_extractor_layer.trainable=False

In [25]:
model=tf.keras.Sequential([feature_extractor_layer, tf.keras.layers.Dropout(0.3), tf.keras.layers.Dense(3, activation='softmax')])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 3843 (15.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [26]:
model.compile(optimizer="adam", loss="categorical_crossentropy" ,metrics=['accuracy'])

In [27]:
history = model.fit(train_dataset, epochs=30, validation_data=val_dataset)

Epoch 1/30
177/177 [==============================] - 44s 168ms/step - loss: 0.4760 - accuracy: 0.3471 - val_loss: 0.3650 - val_accuracy: 0.3425
Epoch 2/30
177/177 [==============================] - 32s 163ms/step - loss: 0.4072 - accuracy: 0.3773 - val_loss: 0.3455 - val_accuracy: 0.4002
Epoch 3/30
177/177 [==============================] - 27s 136ms/step - loss: 0.3814 - accuracy: 0.3921 - val_loss: 0.3312 - val_accuracy: 0.3954
Epoch 4/30
177/177 [==============================] - 27s 137ms/step - loss: 0.3616 - accuracy: 0.4026 - val_loss: 0.3183 - val_accuracy: 0.3833
Epoch 5/30
177/177 [==============================] - 30s 154ms/step - loss: 0.3517 - accuracy: 0.4068 - val_loss: 0.3131 - val_accuracy: 0.3700
Epoch 6/30
177/177 [==============================] - 32s 160ms/step - loss: 0.3437 - accuracy: 0.4146 - val_loss: 0.3110 - val_accuracy: 0.3833
Epoch 7/30
177/177 [==============================] - 27s 135ms/step - loss: 0.3313 - accuracy: 0.4217 - val_loss: 0.3060 - val_ac

In [29]:
test = model.evaluate(test_dataset)

59/59 [==============================] - 6s 105ms/step - loss: 0.2975 - accuracy: 0.3984


In [36]:
predict = model.predict(test_dataset)
class_labels = ['Healthy', 'Diseased']
predicted_class = np.argmax(predict, axis=1)[65]
predicted_class_label = class_labels[predicted_class]

print(predicted_class_label)

59/59 [==============================] - 6s 95ms/step
Diseased


In [ ]:
for test_sample in datasets[1].take(6):
  image,label =test_sample[0], test_sample[1]
  image_scaled